In [1]:
#!pip install -q tf-nightly

In [2]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import time
import os


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
len(text)

1115394

In [5]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [6]:
vocab = sorted(set(text))
len(vocab)

65

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

In [10]:
seq_length = 100
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
for i ,(input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print('Step {:4d}'.format(i))
    print('  input: {} ({:s})'.format(input_idx, repr(idx2char[input_idx])))
    print('  expected output : {} ({:s})'.format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output : 47 ('i')
Step    1
  input: 47 ('i')
  expected output : 56 ('r')
Step    2
  input: 56 ('r')
  expected output : 57 ('s')
Step    3
  input: 57 ('s')
  expected output : 58 ('t')
Step    4
  input: 58 ('t')
  expected output : 1 (' ')


In [15]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [16]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [17]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units, 
           return_sequences=True,
           recurrent_initializer='glorot_uniform',
           stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
vocab_size=len(vocab),
embedding_dim = embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


https://tensorflow.org/tutorials/sequences/images/text_generation_training.png

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, '# (batch_size, seq_len, vocab_size)')

(64, 100, 65) # (batch_size, seq_len, vocab_size)


In [22]:
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


In [23]:
sampled_indices

array([50, 18, 33, 34, 19, 25, 26, 28, 13, 13,  9, 15, 50,  3, 47, 35, 12,
       64, 15, 36, 57, 36, 21, 44, 44, 46, 42, 61, 64, 34, 40, 16, 52, 22,
       53, 20, 44,  5, 26, 35,  6, 58, 18, 19, 56, 29,  3, 51,  6, 56, 24,
       64, 15, 10, 28, 47,  1, 33, 15,  4, 46, 27, 30, 13, 38, 48, 52, 13,
       59, 28, 52, 10, 16,  2, 21, 25, 40, 18,  2, 28, 42, 22, 56, 55,  3,
       45, 43, 34, 19, 30, 47, 57,  2, 38, 36, 24, 56, 63, 11, 62])

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


Input: 
 'ights, thousands will fly.\nFarewell at once, for once, for all, and ever.\n\nBUSHY:\nWell, we may meet '

Next Char Predictions: 
 "lFUVGMNPAA3Cl$iW?zCXsXIffhdwzVbDnJoHf'NW,tFGrQ$m,rLzC:Pi UC&hORAZjnAuPn:D!IMbF!PdJrq$geVGRis!ZXLry;x"


In [25]:
def loss(labels, logits):
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1761174


In [26]:
model.compile(
optimizer = tf.train.AdamOptimizer(),
loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [28]:
EPOCHS = 100

In [29]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/100
174/174 [==============================] - 176s 1s/step - loss: 2.7055
Epoch 2/100
174/174 [==============================] - 176s 1s/step - loss: 1.9463
Epoch 3/100
 86/174 [=============>................] - ETA: 1:27 - loss: 1.7244

KeyboardInterrupt: 

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)


In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))

# Advanced Customized Training

In [ ]:
model = build_model(vocab_size = len(vocab),
                   embedding_dim = embedding_dim, 
                   rnn_units=rnn_units,
                   batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.train.AdamOptimizer()


In [ ]:
EPOCHS=1

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            
            predictions = model(inp)
            loss = tf.losses.sparse_softmax_cross_entropy(target,
                                                         predictions)
            
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if batch_n %100 == 0:
            template = 'Epoch {} Batch {} Loss {:.4f}'
            print(template.format(epoch+1, batch_n, loss))
            
    if (epoch+1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))